Instance type: `m5.8xlarge`

Container using: 24 cores and 120 Gi

In [1]:
import os
import subprocess
import glob

from IPython import get_ipython

ipython = get_ipython()

In [22]:
specie = "pan_onca"

dir_specie = "Ponca_DV_loc"

file_specie = "poncadav2"

dir_mask_specie = "Ponca_DV"

file_mask_specie = "poncamask.tif"

dir_years = "forest_jEquihua_mar"

date_of_processing = "02_06_2021"


In [ ]:
bucket_with_data = "hsi-kale"

input_dir_data = "/shared_volume/input_data"

if not os.path.exists(input_dir_data):
    os.makedirs(input_dir_data)

    
cmd_subprocess = ["aws", "s3", "cp",
                  "s3://" + bucket_with_data,
                  input_dir_data,
                  "--recursive"]

subprocess.run(cmd_subprocess)

```R
## Localidades en shapefile de la especies con los anios
poncaloc<-rgdal::readOGR("/shared_volume/Ponca_DV_loc/","poncadav2")

```

In [ ]:
#
ipython.magic("load_ext rpy2.ipython")
#

string_libraries = """R library(rgdal); library(raster)"""

ipython.magic(string_libraries)

##assignment statements to build string

variable_specie_loc = "specie_loc"

variable_mask_specie = "specie_mask"

string1 = "R " + variable_specie_loc + " <- rgdal::readOGR("

string2 = os.path.join(input_dir_data, dir_specie)

string3 = variable_mask_specie + " <- raster::raster("

string4 = os.path.join(input_dir_data, dir_mask_specie, file_mask_specie)

string_data_input = "".join([string1, "\"", string2, "\",", 
                             "\"", file_specie, "\"",");",
                             string3, "\"", string4, "\"", ")"])

##(end) assignment statements to build string

ipython.magic(string_data_input)

specie_loc = ipython.magic("Rget " + variable_specie_loc)
specie_mask = ipython.magic("Rget " + variable_mask_specie)

```R
poncaloc_transf <- sp::spTransform(poncaloc,
                                   CRSobj = "+proj=lcc +lat_1=17.5 +lat_2=29.5 +lat_0=12 +lon_0=-102 +x_0=2500000 +y_0=0 +datum=WGS84 +units=m +no_defs +ellps=WGS84 +towgs84=0,0,0") 
```

In [21]:
#
ipython.magic("load_ext rpy2.ipython")
print(specie_loc)
ipython.magic("Rpush " + variable_specie_loc)
#

string_libraries = """R library(rgdal)"""

ipython.magic(string_libraries)

##assignment statements to build string

variable_specie_loc_transf = "specie_loc_transf"

string1 = "R " + variable_specie_loc_transf + " <- sp::spTransform("

string2 = "CRSobj = \"+proj=lcc +lat_1=17.5 +lat_2=29.5 +lat_0=12 +lon_0=-102 +x_0=2500000 +y_0=0 +datum=WGS84 +units=m +no_defs +ellps=WGS84 +towgs84=0,0,0\")"

string_transform = "".join([string1, variable_specie_loc, ",",
                            string2])

##(end) assignment statements to build string

ipython.magic(string_transform)

specie_loc_transf = ipython.magic("Rget " + variable_specie_loc_transf)


```R

test_sp <- sp_temporal_data(occs=poncaloc_transf,longitude = "coords.x1",
                            latitude = "coords.x2",sp_year_var="Year",
                            layers_by_year_dir ="/shared_volume/forest_jEquihua_mar/",
                            layers_ext = "*.tif$",reclass_year_data = T)
```

In [4]:
#
ipython.magic("load_ext rpy2.ipython")
print(specie_loc_transf)

ipython.magic("Rpush " + variable_specie_loc_transf)
#
string_libraries = """R library(hsi)"""

ipython.magic(string_libraries)

##assignment statements to build string

variable_test_sp = "test_sp"

string1 = "R " + variable_test_sp + " <- sp_temporal_data(occs="

string2 = "longitude = \"coords.x1\",latitude = \"coords.x2\",sp_year_var=\"Year\",layers_by_year_dir ="

string3 = os.path.join(input_dir_data, dir_years)

string4 = "layers_ext = \"*.tif$\",reclass_year_data = T)"

string_test = "".join([string1, variable_specie_loc_transf, ",",
                       string2, "\"", string3 , "\",",
                       string4])

##(end) assignment statements to build string

ipython.magic(string_test)

test_sp = ipython.magic("Rget " + variable_test_sp)



```R
#Filtrar las localidades que se usaran mediante la mascara

test_sp_mask <- occs_filter_by_mask(test_sp,ponca_mask)

#Limpia localidades duplicadas por anio

test_sp_clean <- clean_dup_by_year(this_species = test_sp_mask,threshold = res(ponca_mask)[1])

e_test <- extract_by_year(this_species=test_sp_clean,layers_pattern="_mar")

```

In [5]:
#
ipython.magic("load_ext rpy2.ipython")


string_libraries = """R library(hsi);library(raster)"""

ipython.magic(string_libraries)

print(test_sp)
print(specie_mask)
ipython.magic("Rpush " + variable_test_sp)
ipython.magic("Rpush " + variable_mask_specie)
#

##assignment statements to build string
variable_test_sp_mask = "test_sp_mask"

string1 = "R " + variable_test_sp_mask + " <- occs_filter_by_mask("

string_filter = "".join([string1, variable_test_sp, ",",
                         variable_mask_specie,
                         ")"])

##(end)assignment statements to build string

ipython.magic(string_filter)

##assignment statements to build string

variable_test_sp_clean = "test_sp_clean"

string1 = "R " + variable_test_sp_clean + " <- clean_dup_by_year(this_species = "

string2 = ", threshold = res("

string3 = ")[1])"

string_clean_test = "".join([string1, variable_test_sp_mask,
                             string2, variable_mask_specie,
                             string3])

##(end)assignment statements to build string

ipython.magic(string_clean_test)

##assignment statements to build string
variable_e_test = "e_test"

string1 = "R " + variable_e_test + " <- extract_by_year(this_species="

string2 = ",layers_pattern=\"_mar\")"

string_extract = "".join([string1, variable_test_sp_clean, string2])

##(end)assignment statements to build string

ipython.magic(string_extract)

e_test = ipython.magic("Rget " + variable_e_test)


```R
best_model_2004 <- find_best_model(this_species = e_test,
                                   cor_threshold = 0.8,
                                   ellipsoid_level = 0.975,
                                   nvars_to_fit = 3,E = 0.05,
                                   RandomPercent = 70,
                                   NoOfIteration = 1000,
                                   parallel = TRUE,
                                   n_cores = 24,
                                   plot3d = FALSE)

```

In [6]:
#
ipython.magic("load_ext rpy2.ipython")
print(e_test)

ipython.magic("Rpush " + variable_e_test)
#
string_libraries = """R library(hsi)"""

ipython.magic(string_libraries)


##assignment statements to build string

variable_best_model_2004 = "best_model_2004"

string1 = "R " + variable_best_model_2004 + " <- find_best_model(this_species ="

string2 = ", cor_threshold = 0.8, ellipsoid_level = 0.975,nvars_to_fit = 3,E = 0.05,RandomPercent = 70,NoOfIteration = 1000,parallel = TRUE,n_cores = 24,plot3d = FALSE)"

string_best_model = "".join([string1, variable_e_test, string2])

##(end)assignment statements to build string


ipython.magic(string_best_model)

best_model_2004 = ipython.magic("Rget " + variable_best_model_2004)


```R

temporal_projection(this_species = best_model_2004,
                    save_dir = "/shared_volume/new_model_parallel/27_05_2021/",
                    sp_mask = ponca_mask,
                    crs_model = NULL,
                    sp_name ="pan_onca",
                    plot3d = FALSE)

```

In [7]:
#
ipython.magic("load_ext rpy2.ipython")

string_libraries = """R library(hsi);library(raster)"""

ipython.magic(string_libraries)

print(best_model_2004)
print(specie_mask)
ipython.magic("Rpush " + variable_best_model_2004)
ipython.magic("Rpush " + variable_mask_specie)
#

dir_results = "/shared_volume/new_model_parallel"

save_dir = os.path.join(dir_results, date_of_processing)

##assignment statements to build string

string1 = "R temporal_projection(this_species = "

string2 = ",save_dir = "

string3 = "sp_mask = "

string4 = ",crs_model = NULL,sp_name ="

string5 = ",plot3d = FALSE)"

string_temporal_proj = "".join([string1, variable_best_model_2004,
                                string2, "\"", save_dir, "\",",
                                string3, variable_mask_specie,
                                string4, "\"", specie, "\"", string5])


##(end)assignment statements to build string


if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
ipython.magic(string_temporal_proj)

#temporal_projection = ipython.magic("Rget temporal_projection")


In [32]:
dir_to_upload = glob.glob(save_dir + '*')[0]

bucket_results = "s3://hsi-kale-results"


bucket_path_uploading = os.path.join(bucket_results, date_of_processing)

cmd_subprocess = ["aws", "s3", "cp",
                  dir_to_upload,
                  bucket_path_uploading,
                  "--recursive"]

subprocess.run(cmd_subprocess)